In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(12, 1), (12, 0), (9, 2), (11, 2), (14, 3), (10, 0), (13, 2), (9, 1)]
[(13, 1), (11, 0), (11, 3), (13, 0), (12, 3), (12, 2), (10, 3), (10, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[1000, 0],
   [0, 0],
   [8.0, 7.522],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.522],
   [2.634, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 1000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.522 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.522 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.634 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer2,514.0,1000.0,NaN,1514.0,2
RandomPlayer,NaN,1000.0,486.0,1486.0,1
YourName,0.0,NaN,0.0,0.0,0


In [12]:
from Test_Players import *

In [13]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ]

compare_all(agent_strategies)

,RandomPlayer,YourName,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,Random_Never_Accuse,Random_Always_Accuse,Mimic,|| Total Wins ||,|| Dominated Players ||
Random_Always_Accuse,392.0,1000.0,1000.0,1000.0,997.0,1000.0,NaN,1000.0,6389.0,6
RandomPlayer,NaN,1000.0,936.0,982.0,883.0,976.0,608.0,990.0,6375.0,7
MyRandom_Real_Cards,117.0,1000.0,646.0,820.0,NaN,977.0,3.0,991.0,4554.0,5
MyRandom_Default,64.0,1000.0,NaN,738.0,354.0,976.0,0.0,988.0,4120.0,4
MyRandom_All_Cards,18.0,1000.0,262.0,NaN,180.0,975.0,0.0,981.0,3416.0,3
Random_Never_Accuse,24.0,1000.0,24.0,25.0,23.0,NaN,0.0,485.0,1581.0,1
Mimic,10.0,1000.0,12.0,19.0,9.0,515.0,0.0,NaN,1565.0,2
YourName,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
